In [2]:
from rdkit import Chem

file_path = 'Tox21_dataset.sdf'
mols = Chem.SDMolSupplier(file_path)

num_molecules = len(mols)
print(f"Number of molecules: {num_molecules}")

molecule_data = []
for i in range(5):  
    mol = mols[i]
    if mol is not None:
        props = mol.GetPropsAsDict()  
        molecule_data.append({
            "Molecule_Index": i,
            "Num_Atoms": mol.GetNumAtoms(),
            "Properties": props
        })
for data in molecule_data:
    print(data)


Number of molecules: 11764
{'Molecule_Index': 0, 'Num_Atoms': 34, 'Properties': {'Formula': 'C27H25ClN6', 'FW': '468.9806 (35.4535+224.2805+209.2465)', 'DSSTox_CID': 25848, 'SR-HSE': 0}}
{'Molecule_Index': 1, 'Num_Atoms': 31, 'Properties': {'Formula': 'C20H6Br4Na2O5', 'FW': '691.8542 (645.8757+22.9892+22.9892)', 'DSSTox_CID': 5234, 'SR-HSE': 0}}
{'Molecule_Index': 2, 'Num_Atoms': 65, 'Properties': {'Formula': 'C47H83NO17', 'FW': '934.1584 (916.1205+18.0379)', 'DSSTox_CID': 28909, 'SR-HSE': 0}}
{'Molecule_Index': 3, 'Num_Atoms': 68, 'Properties': {'Formula': 'C52H54N4O12', 'FW': '927.0048 (329.4575+89.0275+89.0275+329.4575+90.0349)', 'DSSTox_CID': 5513, 'SR-HSE': 1}}
{'Molecule_Index': 4, 'Num_Atoms': 97, 'Properties': {'Formula': 'C66H87N17O14', 'FW': '1342.5025 (1282.4505+60.0520)', 'DSSTox_CID': 26683, 'NR-AR': 0}}


In [3]:
import numpy as np
from rdkit import Chem

def atom_features(atom):
    return np.array([
        atom.GetAtomicNum(),          
        atom.GetDegree(),             
        atom.GetTotalNumHs(),         
        atom.GetImplicitValence(),    
        atom.GetIsAromatic()          
    ], dtype=np.float32)

def bond_features(bond):
    return np.array([
        bond.GetBondTypeAsDouble(),   
        bond.GetIsConjugated(),       
        bond.GetIsAromatic()          
    ], dtype=np.float32)

def process_molecule(mol):
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_features(atom))

    bond_features_list = []
    adjacency_list = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        adjacency_list.append((i, j))
        bond_features_list.append(bond_features(bond))

    return np.array(atom_features_list), np.array(bond_features_list), np.array(adjacency_list)

sample_molecule = mols[0]
atom_features, bond_features, adjacency_list = process_molecule(sample_molecule)

print(atom_features)
print(bond_features)
print(aasfemadjacency_list)


Atom Features:
 [[17.  0.  0.  0.  0.]
 [ 6.  1.  3.  3.  0.]
 [ 7.  3.  0.  0.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 7.  1.  2.  2.  0.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 7.  1.  2.  2.  0.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 7.  2.  0.  0.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 7.  1.  2.  2.  0.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  3.  0.  0.  1.]
 [ 7.  1.  2.  2.  0.]
 [ 6.  2.  1.  1.  1.]
 [ 6.  2.  1.  1.  1.]]
Bond Features:
 [[1.  0.  0. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.  1.  0. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5 1.  1. ]
 [1.5

In [13]:
import torch
from torch_geometric.data import Data

def create_molecule_graph(mol, label_property="SR-HSE"):
    atom_features, bond_features, adjacency_list = process_molecule(mol)

    x = torch.tensor(atom_features, dtype=torch.float)  
    edge_index = torch.tensor(adjacency_list, dtype=torch.long).t().contiguous()  
    edge_attr = torch.tensor(bond_features, dtype=torch.float)  


    label = mol.GetProp(label_property) if mol.HasProp(label_property) else "0"
    y = torch.tensor([int(label)], dtype=torch.long)  


    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

molecule_graphs = [create_molecule_graph(mol) for mol in mols if mol is not None]



[22:47:50] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[22:47:50] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[22:47:50] ERROR: Could not sanitize molecule ending on line 21572
[22:47:50] ERROR: Explicit valence for atom # 3 Cl, 2, is greater than permitted
[22:47:52] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[22:47:52] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[22:47:52] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[22:47:52] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[22:47:52] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[22:47:54] Explicit valence for atom # 2 Si, 8, is greater than permitted
[22:47:54] ERROR: Could not sanitize molecule ending on line 346021
[22:47:54] ERROR: Explicit valence for atom # 

In [14]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(x, batch)

        x = self.fc(x)
        
        return x


In [15]:
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split

molecule_graphs = [create_molecule_graph(mol) for mol in mols if mol is not None]

train_graphs, test_graphs = train_test_split(molecule_graphs, test_size=0.2, random_state=42)
train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)


[22:48:06] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[22:48:06] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[22:48:06] ERROR: Could not sanitize molecule ending on line 21572
[22:48:06] ERROR: Explicit valence for atom # 3 Cl, 2, is greater than permitted
[22:48:08] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[22:48:08] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[22:48:08] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[22:48:08] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[22:48:08] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[22:48:09] Explicit valence for atom # 2 Si, 8, is greater than permitted
[22:48:09] ERROR: Could not sanitize molecule ending on line 346021
[22:48:09] ERROR: Explicit valence for atom # 

In [16]:
model = GCN(num_node_features=5, hidden_channels=64, num_classes=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


In [17]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:  
        optimizer.zero_grad()  
        out = model(data.x, data.edge_index, data.batch)  
        loss = criterion(out, data.y)  
        loss.backward()  
        optimizer.step() 
        total_loss += loss.item() * data.num_graphs  
    return total_loss / len(loader.dataset)

def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)


In [19]:
num_epochs = 50  

for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_acc = test(model, test_loader)
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Epoch: 001, Train Loss: 0.1683, Test Accuracy: 0.9600
Epoch: 002, Train Loss: 0.1533, Test Accuracy: 0.9596
Epoch: 003, Train Loss: 0.1519, Test Accuracy: 0.9605
Epoch: 004, Train Loss: 0.1525, Test Accuracy: 0.9600
Epoch: 005, Train Loss: 0.1525, Test Accuracy: 0.9596
Epoch: 006, Train Loss: 0.1516, Test Accuracy: 0.9600
Epoch: 007, Train Loss: 0.1523, Test Accuracy: 0.9600
Epoch: 008, Train Loss: 0.1511, Test Accuracy: 0.9605
Epoch: 009, Train Loss: 0.1520, Test Accuracy: 0.9605
Epoch: 010, Train Loss: 0.1506, Test Accuracy: 0.9600
Epoch: 011, Train Loss: 0.1515, Test Accuracy: 0.9600
Epoch: 012, Train Loss: 0.1507, Test Accuracy: 0.9600
Epoch: 013, Train Loss: 0.1512, Test Accuracy: 0.9600
Epoch: 014, Train Loss: 0.1506, Test Accuracy: 0.9600
Epoch: 015, Train Loss: 0.1508, Test Accuracy: 0.9600
Epoch: 016, Train Loss: 0.1501, Test Accuracy: 0.9605
Epoch: 017, Train Loss: 0.1500, Test Accuracy: 0.9600
Epoch: 018, Train Loss: 0.1497, Test Accuracy: 0.9605
Epoch: 019, Train Loss: 0.15

In [26]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    for data in loader:
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)
        all_preds.append(pred.cpu().numpy())
        all_labels.append(data.y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Confusion Matrix and Classification Report
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=['Class 0', 'Class 1'])
    ac = accuracy_score(all_labels, all_preds)
    return cm, report, ac

# Evaluate on the test set
cm, report, ac = evaluate_model(model, test_loader)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)
print("Accuracy Score:", ac)


Confusion Matrix:
 [[2258    0]
 [  94    0]]

Classification Report:
               precision    recall  f1-score   support

     Class 0       0.96      1.00      0.98      2258
     Class 1       0.00      0.00      0.00        94

    accuracy                           0.96      2352
   macro avg       0.48      0.50      0.49      2352
weighted avg       0.92      0.96      0.94      2352

Accuracy Score: 0.9600340136054422


C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classificati